# Affected Population by Country (full table, updated daily)
> Displaying the full table of projections of affected populations.

- categories: [overview]
- comments: true
- author: <a href=https://github.com/artdgn/>artdgn</a>
- image: images/affected-pop.png
- permalink: /covid-affected-population/
- toc: true
- hide: false

> Important: This dashboard contains the results of a predictive model that was not built by an epidimiologist.

In [1]:
#hide
import pandas as pd
import covid_helpers

helper = covid_helpers.OverviewData
stylers = covid_helpers.PandasStyling
df = helper.filter_df(helper.table_with_projections())
df.columns

Index(['Cases.new', 'Cases.new.est', 'Cases.new.per100k',
       'Cases.new.per100k.est', 'Cases.total', 'Cases.total.est',
       'Cases.total.per100k', 'Cases.total.per100k.est', 'Continent',
       'Deaths.new', 'Deaths.new.per100k', 'Deaths.total',
       'Deaths.total.per100k', 'Fatality Rate', 'affected_ratio',
       'affected_ratio.est', 'affected_ratio.est.+14d',
       'affected_ratio.est.+14d.err', 'affected_ratio.est.+14d.max',
       'affected_ratio.est.+14d.min', 'affected_ratio.est.+30d',
       'affected_ratio.est.+30d.err', 'affected_ratio.est.+30d.max',
       'affected_ratio.est.+30d.min', 'affected_ratio.est.+60d',
       'affected_ratio.est.+60d.err', 'affected_ratio.est.+60d.max',
       'affected_ratio.est.+60d.min', 'affected_ratio.est.+7d',
       'affected_ratio.est.+7d.err', 'affected_ratio.est.+7d.max',
       'affected_ratio.est.+7d.min', 'affected_ratio.est.+90d',
       'affected_ratio.est.+90d.err', 'affected_ratio.est.+90d.max',
       'affected_ratio.e

In [2]:
#hide_input
from IPython.display import display, Markdown
Markdown(f"***Based on data up to: {pd.to_datetime(helper.dt_today).date().isoformat()}***")

***Based on data up to: 2020-04-12***

## For other details and projections see [main notebook](/notebook-posts/covid-progress-projections/)

## Projected Affected Population percentage

> Countries sorted by number of new cases in last 5 days

- Sorted by number of estimated new cases during the last 5 days.
- Details of estimation and prediction calculations are in [Appendix](#appendix).
- Column definitions:
    - <font size=2><b>Estimated <i>new</i> cases in last 5 days</b>: self explanatory.</font>
    - <font size=2><b>Estimated <i>total</i> affected population percentage</b>: estimated percentage of total population already affected (infected, recovered, or dead).</font>
    - <font size=2><b>Estimated daily case growth rate</b>: percentage daily change in total cases during last 5 days</font>.
    - <font size=2><b>Projected total affected percentage in 14 days</b>: of population.</font>
    - <font size=2><b>Projected total affected percentage in 30 days</b>: of population.</font>
    - <font size=2><b>Reported fatality percentage</b>: reported total deaths divided by total cases.</font>

In [3]:
#hide_input
df_data = df.sort_values('Cases.new.est', ascending=False)
df_pretty = df_data.copy()
df_pretty['affected_ratio.est.+14d'] = stylers.with_errs_ratio(
    df_pretty, 'affected_ratio.est.+14d', 'affected_ratio.est.+14d.err')
df_pretty['affected_ratio.est.+30d'] = stylers.with_errs_ratio(
    df_pretty, 'affected_ratio.est.+30d', 'affected_ratio.est.+30d.err')
df_pretty['growth_rate'] = stylers.with_errs_ratio(df_pretty, 'growth_rate', 'growth_rate_std')

cols = {'Cases.new.est': 'Estimated<br><i>new</i> cases<br>in last<br>5 days',
       'affected_ratio.est': 'Estimated <br><i>total</i><br>affected<br>population<br>percentage',
       'growth_rate': 'Estimated <br> daily case <br> growth rate',
       'affected_ratio.est.+14d': 'Projected<br><i>total</i><br>affected<br>percentage<br>In 14 days',
       'affected_ratio.est.+30d': 'Projected<br><i>total</i><br>affected<br>percentage<br>In 30 days',
       'Fatality Rate': 'Reported <br>fatality <br> percentage',
      }

df_pretty[cols.keys()].rename(cols, axis=1).style\
    .apply(stylers.add_bar, color='#719974',
           s_v=df_data['affected_ratio.est.+14d'], subset=cols['affected_ratio.est.+14d'])\
    .apply(stylers.add_bar, color='#a1afa3',
           s_v=df_data['affected_ratio.est.+30d'], subset=cols['affected_ratio.est.+30d'])\
    .apply(stylers.add_bar, color='#f49d5a',
           s_v=df_data['growth_rate']/0.33, subset=cols['growth_rate'])\
    .bar(subset=cols['Cases.new.est'], color='#b57b17')\
    .bar(subset=cols['affected_ratio.est'], color='#5dad64', vmin=0, vmax=1.0)\
    .bar(subset=cols['Fatality Rate'], color='#420412', vmin=0, vmax=0.1)\
    .applymap(lambda _: 'color: red', subset=cols['Fatality Rate'])\
    .format('<b>{:,.0f}</b>', subset=cols['Cases.new.est'])\
    .format('<b>{:.1%}</b>', subset=[cols['Fatality Rate'], cols['affected_ratio.est']])

,Estimatednew casesin last5 days,Estimated totalaffectedpopulationpercentage,Estimated daily case growth rate,ProjectedtotalaffectedpercentageIn 14 days,ProjectedtotalaffectedpercentageIn 30 days,Reported fatality percentage
Country/Region,,,,,,
US,"493,173",0.5%,6.9% ± 1.1%,1.0% ± 0.1%,1.7% ± 0.4%,4.0%
United Kingdom,"318,325",1.5%,8.7% ± 2.5%,3.2% ± 0.9%,noisy data,12.5%
France,"162,821",1.5%,4.0% ± 1.2%,2.0% ± 0.2%,2.5% ± 0.5%,10.8%
Spain,"147,607",2.2%,3.3% ± 0.7%,2.7% ± 0.1%,3.2% ± 0.3%,10.3%
Italy,"144,236",1.8%,2.9% ± 0.2%,2.2% ± 0.0%,2.4% ± 0.1%,12.7%
Belgium,"63,307",2.3%,6.0% ± 0.7%,3.6% ± 0.2%,5.4% ± 0.7%,12.1%
Turkey,"49,761",0.2%,10.7% ± 1.0%,0.5% ± 0.1%,1.4% ± 0.4%,2.1%
Netherlands,"43,119",1.1%,5.5% ± 0.5%,1.7% ± 0.1%,2.4% ± 0.2%,10.7%
Brazil,"41,902",0.1%,9.3% ± 3.4%,0.1% ± 0.1%,noisy data,5.5%


<a id='examples'></a>

## Interactive plot of Model predictions

For top 20 countries by estimated new cases.

> Tip: Choose a country from the drop-down menu to see the calculations used in the tables above and the dynamics of the model.

In [4]:
#hide_input
sir_plot_countries = df.sort_values('Cases.new.est', ascending=False).head(20).index
_, debug_dfs = helper.table_with_projections(debug_countries=sir_plot_countries)

df_alt = pd.concat([d.reset_index() for d in debug_dfs], axis=0)
covid_helpers.altair_sir_plot(df_alt, sir_plot_countries[0])

alt.LayerChart(...)

## Appendix
<a id='appendix'></a>
[See appendix in main notebook](/notebook-posts/covid-progress-projections/#appendix)